In [107]:
import numpy as np
import pandas as pd
import os
import time
from tqdm import tqdm, trange
import sys
import matplotlib.pyplot as plt

In [ ]:
# variaveis
L = 100 # lado do lattice
n_lagartos = L**2 # lagartos que cabem no lattice
estrategias = ['O', 'Y', 'B'] # estratégias possíveis
a = 2 # ganho em fitness ao vencer
b = 1/a # ganho em fitness ao perder
matriz_payoff = np.array([[1, b, a],
                          [a, 1, b],
                          [b, a, 1]])
index_map = {'O': 0, 'Y': 1, 'B': 2}
n_geracoes = 100
n_pop = 100 # número de populações independentes
prob_mutacao = None # probabilidade de mutação a cada geração
Y_max = 24 # número máximo de vizinhos de Y no teste de sensibilidade
O_max = 24 # número máximo de vizinhos de O na vizinhança adaptativa
output_dir = "C:/Unicamp/mestrado/simulacoes/RPS-python/RPS-POO/outputs/sensibilidade/vizinhosY_e_O/" # diretório para salvar os resultados
os.makedirs(output_dir, exist_ok=True)

In [109]:
class Lagarto:
  def __init__(self, i, j, estrategia, fitness, coord_vizinhos, estrategia_vizinhos, coord_vizinhanca_extendida, estrategia_vizinhanca_extendida, t, n_vizinhos):
    self.i = i # linha
    self.j = j # coluna
    self.estrategia = estrategia
    self.fitness = 0 # inicia com 0 de fitness
    self.coord_vizinhos = [] # lista vazia para adicionar as coordenadas dos vizinhos
    self.estrategia_vizinhos = [] # lista vazia para adicionar as estratégias dos vizinhos
    self.coord_vizinhanca_extendida = []
    self.estrategia_vizinhanca_extendida = []
    self.t = 0 # determina a geracao do lagarto
    self.n_vizinhos = 0 # número de vizinhos

  def calcular_coord_vizinhos(self, L, n_vizinhos_Y, n_vizinhos_O): # obtém as coordenadas dos vizinhos

    lista_vizinhos = []
      
    if self.estrategia == 'B': # estratégia azul mantém número fixo de vizinhos em 8
         # vizinhança de Moore de raio 1 (8 vizinhos)
        for dx in [-1, 0, 1]:
            for dy in [-1, 0, 1]:
                if dx == 0 and dy == 0: # ignora ele mesmo
                    continue
                ni = (self.i + dx) % L # fronteiras periódicas 
                nj = (self.j + dy) % L # fronteiras periódicas 
                lista_vizinhos.append((ni, nj)) # retorna duplex linha (i) e coluna (j) do vizinho

    elif self.estrategia == 'O': # estratégia laranja adapta o número de vizinhos conforme n_vizinhos_O
        # Pega os vizinhos mais próximos (laterais e diagonais), quantidade definida por n_vizinhos_O
        # Pra isso, gera todos os vizinhos possíveis (exceto ele mesmo)
        vizinhos_possiveis = []
        # Começa pegando todos os vizinhos em um quadrado de lado 7 (3 pra cada lado, 49 indivíduos)
        for dx in range(-3, 4): 
            for dy in range(-3, 4):
                if dx == 0 and dy == 0: # ignora ele mesmo
                    continue
                ni = (self.i + dx) % L # fronteiras periódicas
                nj = (self.j + dy) % L # fronteiras periódicas
                vizinhos_possiveis.append(((ni, nj), max(abs(dx), abs(dy)))) # armazena a coordenada dos vizinhos possíveis e a sua distância até o lagarto atual
        # Ordena os vizinhos por distância (mais próximos primeiro)
        vizinhos_possiveis.sort(key=lambda x: x[1])
        # Seleciona os n_vizinhos_O mais próximos
        lista_vizinhos = [coord for coord, _ in vizinhos_possiveis[:n_vizinhos_O]] # pega as coordenadas dos n_vizinhos_O mais próximos

    elif self.estrategia == 'Y': # estratégia amarela adapta o número de vizinhos conforme n_vizinhos_Y
        # faz o mesmo que a função anterior, mas pegando n_vizinhos_Y
        vizinhos_possiveis = []
        for dx in range(-3, 4):
            for dy in range(-3, 4):
                if dx == 0 and dy == 0:
                    continue
                ni = (self.i + dx) % L
                nj = (self.j + dy) % L
                vizinhos_possiveis.append(((ni, nj), max(abs(dx), abs(dy))))
        vizinhos_possiveis.sort(key=lambda x: x[1])
        lista_vizinhos = [coord for coord, _ in vizinhos_possiveis[:n_vizinhos_Y]]

    self.coord_vizinhos = lista_vizinhos # armazena as coordenadas dos vizinhos na variável do lagarto
    
  def obter_estrategia_vizinhos(self, matriz_posicao): # dadas as coordenadas, obtém a estratégia do lagarto que ocupa aquela posição
      self.estrategia_vizinhos = [matriz_posicao[ni, nj] for ni, nj in self.coord_vizinhos] 

  def mutacao(self, prob_mutacao): # função de mutação
    if np.random.rand() < prob_mutacao: # sorteia um valor entre 0 e 1, se for menor que a probabilidade de mutação, o lagarto muda de estratégia
        estrategias_possiveis = [e for e in estrategias if e != self.estrategia] # obtém as estratégias possíveis, exceto a atual
        self.estrategia = np.random.choice(estrategias_possiveis) # escolhe uma nova estratégia aleatoriamente para mutar

  def calcular_n_vizinhos(self): # calcula o número de vizinhos efetivo
      self.n_vizinhos = len(self.estrategia_vizinhos) + len(self.estrategia_vizinhanca_extendida) # soma os vizinhos diretos e os da vizinhança estendida

def calcular_media_vizinhos(lagartos, estrategias): # calcula a média de vizinhos para cada estratégia
    medias = []
    for e in estrategias:
        viz = [lag.n_vizinhos for lag in lagartos if lag.estrategia == e]
        medias.append(np.mean(viz) if len(viz) > 0 else 0) # se não houver lagartos com a estratégia, a média é 0
    return medias # retorna a média de vizinhos para cada estratégia

def ajustar_vizinhos_reciprocos(lagartos): # garante que, se A é vizinho de B, B também é vizinho de A, pois as interações são recíprocas
    mapa = {(l.i, l.j): l for l in lagartos} # dicionário pra acessar lagartos pela posição

    for l in lagartos:
        for (ni, nj) in l.coord_vizinhos: # vai em todos os vizinhos do lagarto atual (l)
            vizinho = mapa[(ni, nj)]
            # se o lagarto atual (l) não estiver na lista de vizinhos do vizinho, adiciona em vizinhanca_extendida
            if (l.i, l.j) not in vizinho.coord_vizinhos:
                vizinho.estrategia_vizinhanca_extendida.append(str(l.estrategia)) 
                vizinho.coord_vizinhanca_extendida.append((l.i, l.j))

In [110]:
def criar_lagartos(n_lagartos, L, estrategias): # define as posições e estratégias dos lagartos no t = 0
  lista_lagartos = []

  # posições iniciais aleatórias
  all_positions = [(i, j) for i in range(L) for j in range(L)] # forma todas as posições possíveis em um lattice
  unique_positions_indices = np.random.choice(len(all_positions), n_lagartos, replace=False) # determina o índice de onde vai ficar cada posição
  unique_positions = [all_positions[i] for i in unique_positions_indices] # basicamente, ele embaralhou as posições

  for g in range(n_lagartos):
    i, j = unique_positions[g] # posição na matriz
    estrategia = np.random.choice(estrategias) # sorteia a estrategia
    lista_lagartos.append(Lagarto(i, j, estrategia, 0, [], [], [], [], 0, 0)) # cria o lagarto com sua posição e identidade
  return lista_lagartos

def calcular_fitness(lagarto, matriz_payoff, index_map, matriz_posicao): # função para calcular o fitness do lagarto
    fitness_total = 0 # inicia no 0
    
    todos_vizinhos = set(lagarto.coord_vizinhos + lagarto.coord_vizinhanca_extendida) # junta todos os vizinhos (normais e estendidos) em um set para evitar que um vizinho seja contado duas vezes
    for ni, nj in todos_vizinhos:
        vizinho_estrat = matriz_posicao[ni, nj] # pega a estratégia do vizinho dadas as suas coordenadas
        if vizinho_estrat is not None:
            fitness_total += matriz_payoff[index_map[lagarto.estrategia], index_map[vizinho_estrat]] # calcula o payoff do lagarto contra o vizinho de acordo com a matriz de payoff e soma ao fitness total
    lagarto.fitness = fitness_total
    return fitness_total

calcular_freq = lambda mat: np.array([np.sum(mat == s) / (L ** 2) for s in ['O', 'Y', 'B']]) # calcula a frequência de cada estratégia no lattice na ordem O, Y, B

In [111]:
def atualizar_lagartos(lagartos): # função que atualiza as estratégias dos lagartos com base no fitness dos vizinhos
    novas_estrategias = {} # Dicionário para armazenar as novas estratégias

    mapa = {(l.i, l.j): l for l in lagartos} # dicionário para acessar lagartos pela posição

    for lagarto in lagartos:
        melhor_estrategia = lagarto.estrategia # inicia com a própria estratégia
        maior_fitness = lagarto.fitness # verifica o fitness do próprio lagarto

        # verifica o fitness dos vizinhos normais
        for (ni, nj) in lagarto.coord_vizinhos:
            vizinho = mapa[(ni, nj)] # usa o dicionário para achar o vizinho
            if vizinho.fitness > maior_fitness: # se o fitness do vizinho for maior que o maior fitness atual
                maior_fitness = vizinho.fitness # atualiza o maior fitness
                melhor_estrategia = vizinho.estrategia # atualiza a melhor estratégia
                # se houver empate de fitness ou for menor, mantém a estratégia atual (não muda)

        # verifica o fitness dos vizinhos extendidos
        for (ni, nj) in lagarto.coord_vizinhanca_extendida:
            vizinho = mapa[(ni, nj)] 
            if vizinho.fitness > maior_fitness:
                maior_fitness = vizinho.fitness
                melhor_estrategia = vizinho.estrategia

        novas_estrategias[(lagarto.i, lagarto.j)] = melhor_estrategia # armazena a nova estratégia no dicionário

    # atualiza as estratégias de todos os lagartos simultaneamente
    for lagarto in lagartos:
        lagarto.estrategia = novas_estrategias[(lagarto.i, lagarto.j)] # evita que a atualização de um lagarto influencie outro na mesma geração (sem sobreposição de geração)
    
    return lagartos

In [112]:

# iniciando a simulação
def simulacao(n_geracoes, L, n_lagartos, estrategias, matriz_payoff, index_map, n_pop, Y_max, O_max, prob_mutacao = None, seed = None):
    matriz_frequencias = np.full((n_geracoes + 1, n_pop, Y_max, O_max, len(estrategias)), np.nan, dtype=float) # cria uma matriz para armazenar as frequências em cada instante dos loops
    matriz_n_vizinhos = np.full((n_geracoes + 1, n_pop, Y_max, O_max, len(estrategias)), np.nan, dtype=float) # cria uma matriz para armazenar vizinhos

    for y in range(1, Y_max+1): # loop para os número de vizinhos de Y 
      print(f"Simulando para y={y}")
      for o in range(1, O_max+1): # loop para o número de vizinhos de O
        print(f"  Simulando para o={o}")
        # faz todas as combinações de Y e O
        for pop in range(n_pop): # loop para cada população independente
          if seed is not None:
            np.random.seed(seed + pop) # coloca uma semente diferente pra cada pop, garantindo independência e reproducibilidade

          frequencias = [] # vai armazenar as frequências ao longo das gerações para essa população
          lista_lagartos = criar_lagartos(n_lagartos, L, estrategias) # cria os lagartos
          matriz_posicao = np.full((L, L), None) # cria uma matriz vazia com None
          
          for lagarto in lista_lagartos:
            matriz_posicao[lagarto.i, lagarto.j] = str(lagarto.estrategia) # cria a matriz de posições de acordo com os lagartos

          frequencias.append(calcular_freq(matriz_posicao)) # calcula a frequência inicial

          for t in range(1, n_geracoes + 1): # loop para cada geração dentro da população
            # determinando os vizinhos
            for lagarto in lista_lagartos:
              lagarto.calcular_coord_vizinhos(L, n_vizinhos_Y = y, n_vizinhos_O = o) # calcula as coordenadas dos vizinhos
              lagarto.obter_estrategia_vizinhos(matriz_posicao) # obtém as estratégias dos vizinhos
              # zera a vizinhança estendida para recalcular a cada lagarto
              lagarto.coord_vizinhanca_extendida = [] 
              lagarto.estrategia_vizinhanca_extendida = [] 

            ajustar_vizinhos_reciprocos(lista_lagartos) # ajusta as vizinhanças recíprocas

            for lagarto in lista_lagartos:
              lagarto.calcular_n_vizinhos() # calcula o número de vizinhos efetivo
          
            matriz_n_vizinhos[t, pop, y-1, o-1, :] = calcular_media_vizinhos(lista_lagartos, estrategias) # calcula a média de vizinhos para cada estratégia e armazena na matriz n_vizinhos
            #print(matriz_n_vizinhos[t, pop, y-1, :]) # debug

            # calculando o fitness
            for lagarto in lista_lagartos:
              calcular_fitness(lagarto, matriz_payoff, index_map, matriz_posicao) # calcula o fitness do lagarto de acordo com seus vizinhos e a matriz de fitness

            # criando a matriz de fitness (parece ser desnecessário)
            matriz_fitness = np.full((L, L), None) # cria uma matriz vazia com None
            for lagarto in lista_lagartos:
              matriz_fitness[lagarto.i, lagarto.j] = float(lagarto.fitness) # coloca os fitness nas posições

            lista_lagartos = atualizar_lagartos(lista_lagartos) # atualiza as estratégias dos lagartos de acordo com o maior fitness dos vizinhos

            if prob_mutacao is not None:
              for lagarto in lista_lagartos:
                lagarto.mutacao(prob_mutacao) # aplica a mutação

            # atualiza a matriz de posição com as novas estratégias e com as mutações
            matriz_posicao = np.full((L, L), None)
            for lagarto in lista_lagartos:
              matriz_posicao[lagarto.i, lagarto.j] = str(lagarto.estrategia)

            frequencias.append(calcular_freq(matriz_posicao)) # calcula a frequência dessa geração e armazena em frequencias

            for lagarto in lista_lagartos:
                lagarto.t += 1 # incrementa a geração do lagarto

            # Armazena as frequências dessa população na matriz de frequências
            
          frequencias = np.array(frequencias)
          for t in range(n_geracoes + 1):
            matriz_frequencias[t, pop, y-1, o-1, :] = frequencias[t]
            
        # Salvar médias por geração e estratégia
        linhas_freq = []
        linhas_viz = []
        for t in range(n_geracoes + 1):
          for idx, strategy in enumerate(estrategias):
              freq_media = np.nanmean(matriz_frequencias[t, :, y-1, o-1, idx])
              freq_std = np.nanstd(matriz_frequencias[t, :, y-1, o-1, idx])
              viz_media = np.mean(matriz_n_vizinhos[t, :, y-1, o-1, idx])
              viz_std = np.std(matriz_n_vizinhos[t, :, y-1, o-1, idx])
              linhas_freq.append({
                  "t": t,
                  "y": y,
                  "o": o,
                  "estrategia": strategy,
                  "frequencia_media": freq_media,
                  "frequencia_std": freq_std
              })
              linhas_viz.append({
                  "t": t,
                  "y": y,
                  "o": o,
                  "estrategia": strategy,
                  "n_vizinhos_media": viz_media,
                  "n_vizinhos_std": viz_std
              })
        # Salva CSV para cada y
        df_freq_y = pd.DataFrame(linhas_freq)
        df_freq_y.to_csv(os.path.join(output_dir, f"frequencias_sensibilidade_y{y}_o{o}.csv"), index=False)
        df_viz_y = pd.DataFrame(linhas_viz)
        df_viz_y.to_csv(os.path.join(output_dir, f"n_vizinhos_y{y}_o{o}.csv"), index=False)


    return matriz_frequencias, matriz_n_vizinhos

freq, n_vizinhos = simulacao(n_geracoes, L, n_lagartos, estrategias, matriz_payoff, index_map, n_pop, Y_max, O_max, prob_mutacao, seed = 1)

Simulando para y=1
  Simulando para o=1


KeyboardInterrupt: 